In [ ]:
import numpy as np
from TMDP import TMDP
from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *
from RiverSwim import RiverSwim
#np.set_printoptions(precision=4)
import math
from constants import *
from policy_utils import *
import torch
import mlflow
from CurriculumPMPO import CurriculumPMPO
import optuna

# Parameters
nS = 200
nA = 2
gamma = .99
mu = np.ones(nS) *1/(nS-2)
mu[0] = mu[nS-1] = 0

xi = np.ones(nS) * 1/(nS)
seed = constants.SEEDS[0]
set_policy_seed(seed)

river = RiverSwim(nS, mu, small=5, large=1000, seed=seed)
tau = .5
tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)

In [ ]:
res = bellman_optimal_q(tmdp.env.P_mat, tmdp.env.reward, tmdp.gamma)
Q = res["Q"]
V = compute_V_from_Q(Q, get_policy(Q))
j_opt = compute_expected_j(V, tmdp.env.mu)
print("optimal performance: ",j_opt)

In [ ]:

def get_or_create_experiment(experiment_name):
    if experiment := mlflow.get_experiment_by_name(experiment_name):
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(experiment_name)


In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
experiment_name = "OPTUNA"
experiment_id = get_or_create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)

In [ ]:
episodes = 4000000

final_temp=1
param_decay=True
lam = 1
batch_size = 32
epochs = 1

eps_ppo = 0.2
eps_model = compute_eps_model(gamma, tau, episodes/10)

In [ ]:

def objective(trial):
    with mlflow.start_run(nested=True):
        global episodes
        # Parameter space optimization
        model_lr = trial.suggest_float("model_lr", 1e-4, 0.5, log=False)
        pol_lr = trial.suggest_float("pol_lr", 1e-4, 0.5, log=False)
        #batch_size = trial.suggest_int("batch_size", 1, 32, log=False)
        #lam = trial.suggest_float("lam", 0., 1., log=False)
        temp = trial.suggest_float("temp", 0.1, 20, log=False)

        # Curriculum class initialization
        curr_PMPO = CurriculumPMPO(tmdp)
        
        tmdp.update_tau(tau) # Reset the teleport rate 
        
        curr_PMPO.train(model_lr, pol_lr, batch_size=batch_size, 
                lam=lam, temp=temp, final_temp=final_temp,
                episodes=episodes, eps_ppo=eps_ppo,
                eps_model=eps_model,
                param_decay=param_decay, epochs=epochs)
        
        avg_performance = np.mean(curr_PMPO.exp_performances[-10:]) # Optimize for the average performance
        avg_reward = np.mean(curr_PMPO.reward_records[-10:])/batch_size
        
        mlflow.log_params({
            "model_lr": model_lr,
            "pol_lr": pol_lr,
            "batch_size": batch_size,
            "lam": lam,
            "temp": temp,
            "final_temp": final_temp,
            "episodes": curr_PMPO.episodes,
        })

        """for i, perf in enumerate(curr_PMPO.exp_performances):
            mlflow.log_metric(f"Epoch {i} Performance ", perf)"""

        mlflow.log_metric("Avg Performance", avg_performance)
        mlflow.log_metric("Avg Reward", avg_reward)

        mlflow.set_tags(tags={
        "Project": "Curriculum Learning",
        "Algorithm": "CurriculumPMPO",
        "Environment": tmdp.env.__class__.__name__,
        "nS": nS,
        "nA": nA,
        "gamma": gamma,
        "mu": tmdp.env.mu,
        "xi": tmdp.xi,
        "seed": seed,
        "tau": tau,
        "episodes": curr_PMPO.episodes,
        })
        curr_PMPO.save_to_mlflow() # Save the model to MLflow

    return avg_performance


In [ ]:
run_name = "CPMPO_RiverSwim_{}".format(nS)
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name, nested=True):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=20)
    
    mlflow.log_params(study.best_params)
    
    # Train and save the best model
    curr_PMPO = CurriculumPMPO(tmdp)
    tmdp.update_tau(tau) # Reset the teleport rate 

    # Get the best parameters
    model_lr = study.best_params["model_lr"]
    pol_lr = study.best_params["pol_lr"]
    #batch_size = study.best_params["batch_size"]
    #lam = study.best_params["lam"]
    temp = study.best_params["temp"]
    
    # Train the best model
    curr_PMPO.train(model_lr, pol_lr, batch_size=batch_size, 
                lam=lam, temp=temp, final_temp=final_temp,
                episodes=episodes, eps_ppo=eps_ppo,
                eps_model=eps_model,
                param_decay=param_decay, epochs=epochs)
    
    mlflow.set_tags(tags={
        "Project": "Curriculum Learning",
        "Algorithm": "CurriculumPMPO",
        "Environment": tmdp.env.__class__.__name__,
        "nS": nS,
        "nA": nA,
        "gamma": gamma,
        "mu": tmdp.env.mu,
        "xi": tmdp.xi,
        "seed": seed,
        "tau": tau,
        "episodes": episodes,
        "eps_model": eps_model,
        "eps_ppo": eps_ppo,
        })

    curr_PMPO.save_to_mlflow() # Save the best model to MLflow

In [ ]:
study.best_params

In [ ]:

delta_pol = get_policy(Q)-get_policy(get_softmax_policy(curr_PMPO.thetas[-10]))
print(delta_pol)

r_s_a = compute_r_s_a(tmdp.env.P_mat, tmdp.env.reward)


V_curr = compute_V_from_Q(Q, get_policy(get_softmax_policy(curr_PMPO.theta)))

j_curr = compute_expected_j(V_curr, tmdp.env.mu)
j_opt = compute_expected_j(V, tmdp.env.mu)
print("optimal performance: ",j_opt)
print("curriculum performance: ", j_curr)

print(get_softmax_policy(curr_PMPO.theta))

In [ ]:
import matplotlib.pyplot as plt
reward_records = curr_PMPO.reward_records
# Generate recent 50 interval average
average_reward = []
scaled_reward = []
for idx in range(len(reward_records)):
    avg_list = np.empty(shape=(1,), dtype=int)
    scaled_reward.append(reward_records[idx])
    if idx < 500:
        avg_list = reward_records[:idx+1]
    else:
        avg_list = reward_records[idx-499:idx+1]
    average_reward.append(np.mean(avg_list))
# Plot
#plt.plot(scaled_reward)
plt.plot(average_reward[:-1])

In [ ]:
import matplotlib.pyplot as plt
exp_performances = curr_PMPO.exp_performances
# Generate recent 50 interval average
avg_performances = []
for idx in range(len(exp_performances)):
    avg_list = np.empty(shape=(1,), dtype=int)
    if idx < 50:
        avg_list = exp_performances[:idx+1]
    else:
        avg_list = exp_performances[idx-49:idx+1]
    avg_performances.append(np.mean(avg_list))
# Plot
plt.plot(exp_performances)
plt.plot(avg_performances)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

exp_performances = curr_PMPO.exp_performances
# Generate recent 50 interval average
avg_performances = []
std_errors = []

for idx in range(len(exp_performances)):
    if idx < 500:
        avg_list = exp_performances[:idx+1]
    else:
        avg_list = exp_performances[idx-499:idx+1]

    average = np.mean(avg_list)
    std_dev = np.std(avg_list)
    n_samples = len(avg_list)
    sem = std_dev / np.sqrt(n_samples)  # Standard Error of the Mean
    std_errors.append(sem)

    avg_performances.append(average)

# Compute the upper and lower bounds for the 95% confidence interval
ci = 1.96  # Z-score for 95% confidence
upper_bound = [avg + ci * sem for avg, sem in zip(avg_performances, std_errors)]
lower_bound = [avg - ci * sem for avg, sem in zip(avg_performances, std_errors)]

# Plot
plt.figure(figsize=(10, 5))
#plt.plot(exp_performances, label='Exp Performances')
plt.plot(avg_performances, label='Moving Average', color='r')
plt.fill_between(range(len(avg_performances)), lower_bound, upper_bound, color='r', alpha=0.2, label='95% Confidence Interval')
plt.legend()
plt.title('Experiment Performances with Moving Average and Confidence Interval')
plt.xlabel('Index')
plt.ylabel('Performance')
plt.show()
